# Binary Knapsack Problem: A Genetic Algorithm

A very basic genetic algorithm for solving the binary knapsack problem.

In [66]:
import numpy as np

In [463]:
maxVol = 100
numProducts = 200

In [464]:
np.random.seed(42)
volumes = np.random.uniform(0,2, numProducts)
weights = np.random.uniform(0,5, numProducts)
costs = np.random.uniform(0, 5, numProducts)
values = costs + np.random.uniform(0, 10, numProducts)

In [465]:
# Defining classes
class BK:
    def __init__(self, maxVol, volumes, weights, costs, values):
        self.maxVol = maxVol
        self.volumes = volumes
        self.weights = weights
        self.costs = costs
        self.values = values
        
    def get_maxVol(self):
        return self.maxVol
    
    def get_volumes(self):
        return self.volumes
    
    def get_weights(self):
        return self.weights
    
    def get_costs(self):
        return self.costs
    
    def get_values(self):
        return self.values
        
        
class solution:
    def __init__(self):
        self.order = np.array(np.random.permutation(range(numProducts)))
        self.alpha = np.random.random()
        self.beta = np.random.random()
        
    def get_order(self):
        return self.order
    
    def set_order(self, order):
        self.order = order
    
    def get_recombination_probability(self):
        return self.alpha
    
    def get_mutation_probability(self):
        return self.beta

In [466]:
def packagingOrder(BK, solution):
    remainingVolume = BK.get_maxVol()
    order = solution.get_order()
    volumes = BK.get_volumes()
    
    items = []    
    j = 0
    
    while(True):
        item = order[j]
        volume = volumes[j]
        
        if volume <= remainingVolume:
            items.append(item)
            remainingVolume -= volumes[j]
            j += 1
        else:
            break
            
    return items

In [467]:
def totalWeight(BK, items):
    ww = BK.get_weights()
    totalWeights = ww[items].sum()
    
    return totalWeights

def totalNum(items):
    return len(items)

def totalCosts(BK, items):
    cc = BK.get_costs()
    totalCosts = cc[items].sum()
    
    return totalCosts

def totalValue(BK, items):
    vv = BK.get_values()
    totalValues = vv[items].sum()
    
    return totalValues

In [468]:
def initializePopulation(lam):
    population = []
    for i in range(lam):
        population.append(solution())
    return population

In [469]:
def individualFitness(BK, solution):
    order = packagingOrder(BK, solution)
    
    tW = totalWeight(BK, order)
    tN = totalNum(order)
    tC = totalCosts(BK, order)
    tV = totalValue(BK, order)
    
    fitness = tW + tN - tC + tV
    
    return fitness


def populationFitness(population, BK):
    individualScores = []
    for solution in population:
        individualScores.append(individualFitness(BK, solution))
    
    return individualScores

In [470]:
# Def Mutation operators 

In [471]:
def scramble_mutation(solution):
    order = solution.get_order()
    inds = np.random.choice(numProducts, 2, replace=False)
    first_ind = min(inds)
    second_ind = max(inds)
    temp = order[first_ind:second_ind]
    temp = np.random.permutation(temp)
    order[first_ind:second_ind] = temp
    solution.set_order(order)

In [472]:
def roll_mutation(solution):
    solution.set_order(np.roll(solution.get_order(), np.random.choice(numProducts)))

In [473]:
# Def Recombination operators 

In [474]:
def PMX_crossover(parent1, parent2):
    order1 = parent1.get_order()
    order2 = parent2.get_order()
    
    inds = np.random.choice(numProducts, 2, replace=False)
    fci = min(inds)
    sci = max(inds)
    
    offspring_order = np.zeros(numProducts, dtype=int)
    
    # Step 1
    parent_segment = order1[fci:sci+1]
    offspring_order[fci:sci+1] = parent_segment
    parent2_segment = order2[fci:sci+1]
    
    # Step 2
    for elem in order2[fci:sci+1]:
        if elem not in parent_segment:
            p2_elem = elem
            while(True):
                help_elem = order1[np.where(order2 == p2_elem)]
                if help_elem not in parent2_segment:
                    offspring_order[np.where(order2 == help_elem)] = elem
                    break
                else:
                    p2_elem = order1[np.where(order2 == p2_elem)]
                    
    # Step 3
    for elem in order2:
        if elem not in offspring_order:
            offspring_order[np.where(order2 == elem)] = elem
            
    
    child = solution()
    child.set_order(offspring_order)
    return child

In [475]:
# K_tournament selection
def k_tour(BK, k, population):
    inds = []
    ind_fits = []
    for i in range(k):
        index = np.random.randint(0, lam)
        inds.append(index)
        ind_fits.append(individualFitness(BK, population[index]))
    
    best_score = max(ind_fits)
    win_index = inds[ind_fits.index(best_score)]
    winner = population[win_index]
    return winner

In [476]:
# Elimination
def elimination(population):
    fitnesses = populationFitness(population, binknap)
    x = np.array(fitnesses).argsort()[-lam:][::-1]
    popul = population[x]
    return popul

In [477]:
# Initiate classes 
binknap = BK(maxVol, volumes, weights, costs, values)
ordering = solution()

In [478]:
# Initialize four populations
lam = 100
mu = 500
population = initializePopulation(lam)
j = 0
np.mean(populationFitness(population, binknap))

mean_fits = []
max_fits = []

In [479]:
while (j < 200):
    parents = []

    for i in range(mu):
        parents.append(k_tour(binknap, 3, population))
    
    midpoint = mu // 2

    offspring = []

    for i in range(midpoint):
        child1 = PMX_crossover(parents[i], parents[midpoint+i])
        child2 = PMX_crossover(parents[midpoint+i], parents[i])
        offspring.append(child1)
        offspring.append(child2)
        
    for individual in offspring:
        if j % 2 == 0:
            scramble_mutation(individual)
        else:
            roll_mutation(individual)
            
    
    jP = np.append(population, offspring, axis=0)
    population = elimination(jP)
    
    fits = populationFitness(population, binknap)
    mean_fits.append(np.mean(fits))
    max_fits.append(np.max(fits))
    
    print(mean_fits[-1])
    print(max_fits[-1])
    print(j)
    print('')
    
    j+= 1

971.1803770139802
1000.601551666673
0

981.733697904743
1009.520436402853
1

1002.7695121040041
1025.6355934220362
2

1006.547317847396
1025.6355934220362
3

1021.430284144489
1044.8302358253181
4

1025.8273958916473
1052.7844866984133
5

1043.7738964156538
1072.1409398512596
6

1045.3605514288686
1072.1409398512596
7

1057.1010213802722
1079.8582370713534
8

1058.1310411021532
1079.8582370713534
9

1069.9056005843602
1092.0291132361867
10

1071.3237877359309
1092.0291132361867
11

1080.9986361463848
1096.8658058305996
12

1081.8300039500255
1096.8658058305996
13

1090.1216504934773
1108.8673478194326
14

1090.4881344253843
1108.8673478194326
15

1097.1389803098623
1116.4997934359499
16

1097.682050564123
1116.4997934359499
17

1105.929471531139
1117.2035528397469
18

1106.1545654716267
1117.2035528397469
19

1111.2389698890088
1118.1706337047542
20

1111.4416240828366
1118.1706337047542
21

1115.9446293298993
1124.6393745645046
22

1116.1207815374362
1124.6393745645046
23

1119.924850

1194.604422372101
1195.9696895744044
197

1194.722484033438
1195.9696895744044
198

1194.7446348648111
1195.9696895744044
199

